# MNIST CNN

In [1]:
# 导入
import sys

sys.path.append("E:/dataFiles/github/MFlow")

In [2]:
# 数据生成
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder

# TODO: 不使用sklearn的OneHotEncoder

# 加载MNIST数据集
xs, labels = fetch_openml("mnist_784", version=1, return_X_y=True)
xs, labels  = xs[:1000] / 255, labels.astype("uint8")[:1000]
# 转换为one-hot
ohe = OneHotEncoder(sparse=False)
ys = ohe.fit_transform(labels.reshape(-1, 1))

print(xs.shape, ys.shape)
print(ys[0])

(1000, 784) (1000, 10)
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [3]:
# 训练
from mflow import core, ops, opts, lays

# 超参数
lr = 0.005
epoch = 10  # 30太久，节约时间
batch_size = 32

img_size = (28, 28)

with core.NameScope("MNISTCNN"):
    # 初始化变量
    x = core.Variable(size=img_size, trainable=False)
    y = core.Variable(size=(10, 1), trainable=False)
    # 模型定义
    conv1 = lays.Conv([x], img_size, 3, (5, 5), "ReLU")
    poling1 = lays.Pooling(conv1, (3, 3), (2, 2))
    conv2 = lays.Conv(poling1, (14, 14), 3, (3, 3), "ReLU")  # 28 / 2
    poling2 = lays.Pooling(conv2, (3, 3), (2, 2))
    fc1 = lays.Linear(ops.Concat(*poling2), 147, 120, "ReLU")  # 3 * 7 * 7
    pred = lays.Linear(fc1, 120, 10, None)
    predicter = ops.SoftMax(pred)
    loss = ops.loss.CrossEntropyWithSoftMax(pred, y)
    adam = opts.Adam(core.DefaultGraph, loss, lr)
    # 开始训练
    for ep in range(1, epoch + 1):
        bs_idx = 0  # 批次计数
        # 这是一个epoch的过程
        for i, (feat, one_hot) in enumerate(zip(xs, ys)):
            x.setValue(np.mat(feat).reshape(img_size))
            y.setValue(np.mat(one_hot).T)
            adam.step()
            bs_idx += 1
            if bs_idx == batch_size:
                adam.update()
                bs_idx = 0
        # 一个epoch完成后进行评估
        preds = []
        for feat in xs:
            x.setValue(np.mat(feat).reshape(img_size))
            predicter.forward()
            preds.append(predicter.value.A.ravel())  # 结果
        preds = np.array(preds).argmax(axis=1)
        acc = (labels == preds).astype("uint8").sum() / len(xs)
        print("Epoch: {:d}, acc: {:.3f}.".format(ep, acc))

Epoch: 1, acc: 0.228.
Epoch: 2, acc: 0.801.
Epoch: 3, acc: 0.893.
Epoch: 4, acc: 0.920.
Epoch: 5, acc: 0.949.
Epoch: 6, acc: 0.958.
Epoch: 7, acc: 0.963.
Epoch: 8, acc: 0.974.
Epoch: 9, acc: 0.967.
Epoch: 10, acc: 0.969.
